## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/2022_Genea_Challenge/improved-diffusion'

/content/drive/.shortcut-targets-by-id/1IbbuHNJgAKic--1q5oPVztWy2XGht7Zg/2022_Genea_Challenge/improved-diffusion


In [3]:
!ls

cifar_test		     LICENSE				 run.py
cifar_train		     README.md				 scripts
datasets		     results				 setup.py
improved_diffusion	     Rodolfo_RunImprovedDiffusion.ipynb
improved_diffusion.egg-info  RunImprovedDiffusion.ipynb


In [3]:
%%capture
!pip install -e .
# Library for distributed training... Installed only for passing the imports.
!pip install mpi4py

## Load data drive

In [4]:
%cd /content/
!mkdir trn
%cd /content/trn
!unzip /content/drive/MyDrive/2022_Genea_Challenge/dataset_v1/trn/processed_30overlap/input.zip > /content/tmp
!unzip /content/drive/MyDrive/2022_Genea_Challenge/dataset_v1/trn/processed_30overlap/label.zip > /content/tmp

/content
/content/trn


In [5]:
cd '/content/drive/MyDrive/2022_Genea_Challenge/improved-diffusion'

/content/drive/.shortcut-targets-by-id/1IbbuHNJgAKic--1q5oPVztWy2XGht7Zg/2022_Genea_Challenge/improved-diffusion


## Train

In [6]:
%%writefile run.py

#!usr/bin/bash python

import argparse
from improved_diffusion import dist_util, logger
from improved_diffusion.gesture_datasets import load_data
from improved_diffusion.resample import create_named_schedule_sampler
from improved_diffusion.script_util import (
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    args_to_dict,
    add_dict_to_argparser,
)
from improved_diffusion.train_util import TrainLoop

def create_argparser():
    defaults = dict(
        data_dir="",
        schedule_sampler="uniform",
        lr=1e-4,
        weight_decay=0.0,
        lr_anneal_steps=0,
        batch_size=1,
        microbatch=-1,  # -1 disables microbatches
        ema_rate="0.9999",  # comma-separated list of EMA values
        log_interval=10,
        save_interval=1000,
        resume_checkpoint="",
        use_fp16=False,
        fp16_scale_growth=1e-3,
    )
    defaults.update(model_and_diffusion_defaults())
    parser = argparse.ArgumentParser()
    add_dict_to_argparser(parser, defaults)
    return parser

def main():

    args = create_argparser().parse_args()
    print(args)
    dist_util.setup_dist()
    logger.configure(dir = '/content/drive/MyDrive/2022_Genea_Challenge/improved-diffusion/results')

    logger.log("creating model and diffusion...")
    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    model.to(dist_util.dev())
    schedule_sampler = create_named_schedule_sampler(args.schedule_sampler, diffusion)

    logger.log("creating data loader...")
    
    data = load_data(
    data_dir='/content/trn',
    batch_size=16,
    motion_features = 249
    )

    logger.log("training...")
    TrainLoop(
        model=model,
        diffusion=diffusion,
        data=data,
        batch_size=args.batch_size,
        microbatch=args.microbatch,
        lr=args.lr,
        ema_rate=args.ema_rate,
        log_interval=args.log_interval,
        save_interval=args.save_interval,
        resume_checkpoint=args.resume_checkpoint,
        use_fp16=args.use_fp16,
        fp16_scale_growth=args.fp16_scale_growth,
        schedule_sampler=schedule_sampler,
        weight_decay=args.weight_decay,
        lr_anneal_steps=args.lr_anneal_steps,
    ).run_loop()

if __name__ == "__main__":
    main()

Overwriting run.py


In [7]:
FLAGS_UNUSED ="--image_size 64 --num_channels 128 --num_res_blocks 3 --class_cond True --use_checkpoint False  --attention_resolutions 16,8 --num_heads 4 --num_heads_upsample -1 --use_scale_shift_norm True  --dropout 0.0"
DIFFUSION_FLAGS = "--diffusion_steps 1000 --learn_sigma False --sigma_small False --noise_schedule cosine --use_kl True --predict_xstart False --rescale_timesteps False --rescale_learned_sigmas False"
TRAIN_FLAGS="--schedule_sampler loss-second-moment --lr 1e-4 --weight_decay 0.0 --lr_anneal_step 0 --batch_size 16 --microbatch -1 --ema_rate 0.9999  --log_interval 10 --save_interval 1000 --use_fp16 False --fp16_scale_growth 1e-3"

In [8]:
%run run.py $FLAGS_UNUSED $DIFFUSION_FLAGS $TRAIN_FLAGS

Streaming output truncated to the last 5000 lines.
| step      | 2.13e+04 |
------------------------
------------------------
| grad_norm | 26.8     |
| loss      | 538      |
| loss_q0   | 541      |
| loss_q1   | 408      |
| loss_q2   | 371      |
| loss_q3   | 540      |
| samples   | 3.41e+05 |
| step      | 2.13e+04 |
------------------------
------------------------
| grad_norm | 26.7     |
| loss      | 540      |
| loss_q0   | 504      |
| loss_q3   | 540      |
| samples   | 3.41e+05 |
| step      | 2.13e+04 |
------------------------
------------------------
| grad_norm | 26.7     |
| loss      | 539      |
| loss_q0   | 535      |
| loss_q1   | 403      |
| loss_q3   | 539      |
| samples   | 3.41e+05 |
| step      | 2.13e+04 |
------------------------
------------------------
| grad_norm | 26.7     |
| loss      | 539      |
| loss_q0   | 532      |
| loss_q3   | 539      |
| samples   | 3.42e+05 |
| step      | 2.14e+04 |
------------------------
------------------------

OSError: ignored

# Sampling

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd '/content/drive/MyDrive/2022_Genea_Challenge/improved-diffusion'

In [ ]:
%%capture
!pip install -e .
!pip install mpi4py

In [ ]:
%%writefile sample.py

"""
Generate a large batch of image samples from a model and save them as a large
numpy array. This can be used to produce samples for FID evaluation.
"""

#! usr/bin/bash python
import argparse
import os

import numpy as np
import torch as th
import torch.distributed as dist

from improved_diffusion import dist_util, logger
from improved_diffusion.script_util import (
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    add_dict_to_argparser,
    args_to_dict,
)


def main():
    args = create_argparser().parse_args()

    dist_util.setup_dist()
    logger.configure()

    logger.log("creating model and diffusion...")
    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    model.load_state_dict(
        dist_util.load_state_dict(args.model_path, map_location="cpu")
    )
    model.to(dist_util.dev())
    model.eval()

    logger.log("sampling...")
    
    all_audio = []
    all_samples = []
    audio_path = '/content/drive/MyDrive/2022_Genea_Challenge/dataset_v1/val/processed_30overlap/input/0/'
    chunks = os.listdir(audio_path)
    for chunk in chunks:
        all_audio.append(th.tensor(np.load(audio_path+chunk).astype(np.float32)).unsqueeze(0))
    
    #for idx in range(len(all_audio)):
    for idx in range(10):
        audio = all_audio[idx]
        model_kwargs = {'in_audio':audio}
        sample_fn = (
            diffusion.p_sample_loop
        )
        sample = sample_fn(
            model,
            (1, 61, 249),
            clip_denoised=args.clip_denoised,
            model_kwargs=model_kwargs,
        )
        print(idx)
        all_samples.append(sample)
    
    gen_pose_seq = th.cat(all_samples, dim = 1).squeeze(0).cpu().detach().numpy()
    np.save('generate_pose', gen_pose_seq)
    logger.log("sampling complete")


def create_argparser():
    defaults = dict(
        clip_denoised=True,
        num_samples=1,
        batch_size=1,
        use_ddim=False,
        model_path='/content/drive/MyDrive/2022_Genea_Challenge/improved-diffusion/results/model027000.pt',
    )
    defaults.update(model_and_diffusion_defaults())
    parser = argparse.ArgumentParser()
    add_dict_to_argparser(parser, defaults)
    return parser


if __name__ == "__main__":
    main()

In [ ]:
FLAGS_UNUSED ="--image_size 64 --num_channels 128 --num_res_blocks 3 --class_cond True --use_checkpoint False  --attention_resolutions 16,8 --num_heads 4 --num_heads_upsample -1 --use_scale_shift_norm True  --dropout 0.0"
DIFFUSION_FLAGS = "--diffusion_steps 1000 --learn_sigma False --sigma_small False --noise_schedule cosine --use_kl True --predict_xstart False --rescale_timesteps False --rescale_learned_sigmas False"
TRAIN_FLAGS="--schedule_sampler loss-second-moment --lr 1e-4 --weight_decay 0.0 --lr_anneal_step 0 --batch_size 16 --microbatch -1 --ema_rate 0.9999  --log_interval 10 --save_interval 1000 --use_fp16 False --fp16_scale_growth 1e-3"

In [ ]:
!python sample.py $FLAGS_UNUSED $DIFFUSION_FLAGS